In [1]:
import os
import gc
import re
import json
import pickle
import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd
pd.options.display.max_columns = None

from typing import Union

import seaborn
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
%config InlineBackend.figure_formats = {'png', 'retina'}

from scipy.stats import linregress

from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import lightgbm as lgb

from v01005_Baseline import *

In [11]:
VERSION = 'v01005'
SEED = 42

In [4]:
train = pd.read_pickle('/Users/rui/Documents/repositories/m5_forecasting_accuracy/v01000/features/add_sales_train.pkl')

In [5]:
train_data, eval_data, submit_data = split_train_eval_submit(train)

In [6]:
submit_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,quarter,weekofyear,day,dayofweek,is_quarter_end,is_quarter_start,is_month_end,is_month_start,is_weekend,sell_price,sell_price_rate_by_wm_yr_wk__item_id,sell_price_rate_by_wm_yr_wk__area__item_id,sell_price_momentum,sales_lag_t28,sales_lag_t29,sales_lag_t30,sales_lag_t32,sales_lag_t33,sales_lag_t34,sales_rolling_STD_t7,sales_rolling_STD_t30,sales_rolling_STD_t60,sales_rolling_STD_t90,sales_rolling_STD_t180,sales_rolling_MEAN_t7,sales_rolling_MEAN_t30,sales_rolling_MEAN_t60,sales_rolling_MEAN_t90,sales_rolling_MEAN_t180,sales_rolling_MIN_t7,sales_rolling_MIN_t30,sales_rolling_MIN_t60,sales_rolling_MAX_t7,sales_rolling_MAX_t30,sales_rolling_MAX_t60,sales_rolling_ZeroRatio_t7,sales_rolling_ZeroCount_t7,sales_rolling_ZeroRatio_t14,sales_rolling_ZeroCount_t14,sales_rolling_ZeroRatio_t30,sales_rolling_ZeroCount_t30,sales_rolling_ZeroRatio_t60,sales_rolling_ZeroCount_t60,sales_rolling_SKEW_t30,sales_rolling_KURT_t30
58327370,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,d_1914,0,2016-04-25,11613,4,19,2,3,1,0,0,0,2,17,25,0,0,0,0,0,0,8.382812,1.004883,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0,0.689941,0.927734,1.081055,1.125000,0.996094,1.142578,1.033203,0.983398,0.944336,0.772461,0.0,0.0,0.0,2.0,3.0,4.0,0.856934,1.0,0.928711,1.0,0.700195,9.0,0.616699,23.0,0.763184,0.018387
58327371,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,d_1914,0,2016-04-25,11613,4,19,2,3,1,0,0,0,2,17,25,0,0,0,0,0,0,3.970703,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.488037,0.479492,0.436768,0.410400,0.777832,0.714355,0.333252,0.250000,0.211060,0.377686,0.0,0.0,0.0,1.0,1.0,1.0,0.714355,2.0,0.357178,9.0,0.333252,20.0,0.250000,45.0,0.745117,-1.553711
58327372,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,d_1914,0,2016-04-25,11613,4,19,2,3,1,0,0,0,2,17,25,0,0,0,0,0,0,2.970703,1.000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.488037,1.166992,0.888672,0.766602,0.996582,0.285645,0.500000,0.416748,0.344482,0.633301,0.0,0.0,0.0,1.0,6.0,6.0,0.285645,5.0,0.214233,11.0,0.300049,21.0,0.316650,41.0,3.904297,17.734375
58327373,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,d_1914,0,2016-04-25,11613,4,19,2,3,1,0,0,0,2,17,25,0,0,0,0,0,0,4.640625,1.000000,1.0,1.0,0.0,6.0,6.0,5.0,3.0,1.0,2.708984,2.076172,1.954102,1.886719,1.743164,3.000000,1.966797,2.099609,2.033203,1.900391,0.0,0.0,0.0,6.0,6.0,6.0,0.714355,2.0,0.785645,3.0,0.666504,10.0,0.733398,16.0,0.791016,-0.692871
58327374,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,d_1914,0,2016-04-25,11613,4,19,2,3,1,0,0,0,2,17,25,0,0,0,0,0,0,2.880859,1.015625,1.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,1.112305,0.959473,1.170898,1.116211,1.180664,0.714355,1.099609,1.133789,0.966797,1.171875,0.0,0.0,0.0,3.0,3.0,5.0,0.428467,4.0,0.643066,5.0,0.700195,9.0,0.633301,22.0,0.541992,-0.524414


In [7]:
target = 'sales'
cols_to_drop = ['id', 'wm_yr_wk', 'd', 'date'] + [target]
feature = train.columns.tolist()
feature = [f for f in feature if f not in cols_to_drop]

In [12]:
def evaluation_model(eval_data, feature):
    lgbm_model = load_pickle(f'result/model/{VERSION}.pkl')
    preds = lgbm_model.predict(eval_data[feature].values, expm1=False)
    metric_scores = {}
    metric_scores['RMSE'] = mean_squared_error(eval_data['sales'].values, preds, squared=False)
    metric_scores['WRMSSE'] = estimate_wrmsse(eval_data, preds)
    for metric, score in metric_scores.items():
        print(f'{metric}: {score}')
    # Dump metric_scores as Json file.
    with open(f'result/score/{VERSION}.json', 'w') as file:
        json.dump(metric_scores, file, indent=4)
    return metric_scores

metric_scores = evaluation_model(eval_data, feature)

/Users/rui/Documents/repositories/m5_forecasting_accuracy/v01000/v01005_Baseline.py:695: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data['pred'] = preds


RMSE: 2.1885556369028643
WRMSSE: 0.5061008381009501


In [19]:
def create_submission_file(submit_data, feature, score):
    submission = pd.read_pickle('../data/reduced/sample_submission.pkl')['id'].to_frame()
    lgbm_model = load_pickle(f'result/model/{VERSION}.pkl')

    submit_data['pred'] = lgbm_model.predict(submit_data[feature].values, expm1=False)
    sub_validation = pd.pivot(submit_data, index='id', columns='date', values='pred').reset_index()
    sub_validation.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    submission = pd.merge(submission, sub_validation, how='left', on='id')
    submission.fillna(0, inplace=True)
    submission.to_csv(f'submit/{VERSION}_{score:.04f}.csv.gz', index=False, compression='gzip')

    print('Submit DataFrame:', submission.shape)
    print(submission.head())
    
    return submission

In [20]:
create_submission_file(submit_data, feature, metric_scores['WRMSSE'])

<ipython-input-19-5add11f5d50c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit_data['pred'] = lgbm_model.predict(submit_data[feature].values, expm1=False)


Submit DataFrame: (60980, 29)
                              id        F1        F2        F3        F4  \
0  HOBBIES_1_001_CA_1_validation  0.852392  0.688131  0.638466  0.618083   
1  HOBBIES_1_002_CA_1_validation  0.447386  0.405067  0.398567  0.342537   
2  HOBBIES_1_003_CA_1_validation  0.485245  0.448930  0.443987  0.439714   
3  HOBBIES_1_004_CA_1_validation  1.953827  1.650163  1.582948  1.449167   
4  HOBBIES_1_005_CA_1_validation  0.973892  0.852962  0.938600  1.009142   

         F5        F6        F7        F8        F9       F10       F11  \
0  0.713469  0.918846  0.997559  0.681548  0.823610  0.819872  0.782909   
1  0.399995  0.452769  0.493148  0.336008  0.308408  0.274372  0.225396   
2  0.487282  0.614145  0.746810  0.489232  0.467438  0.433062  0.404051   
3  1.882337  2.292001  2.373877  1.683620  1.729427  1.583586  1.363456   
4  1.059734  1.381942  1.712063  1.207382  1.142862  1.006309  0.848956   

        F12       F13       F14       F15       F16       F17 

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.852392,0.688131,0.638466,0.618083,0.713469,0.918846,0.997559,0.681548,0.823610,0.819872,0.782909,0.958344,1.220654,1.158866,0.942417,0.816819,0.793497,0.768756,0.915941,1.219204,1.119833,0.858488,0.745328,0.749085,0.848337,0.863490,1.049259,1.110701
1,HOBBIES_1_002_CA_1_validation,0.447386,0.405067,0.398567,0.342537,0.399995,0.452769,0.493148,0.336008,0.308408,0.274372,0.225396,0.295728,0.370444,0.332283,0.237483,0.210969,0.205713,0.205713,0.260990,0.340830,0.355583,0.239636,0.209479,0.245122,0.235682,0.276393,0.357421,0.361587
2,HOBBIES_1_003_CA_1_validation,0.485245,0.448930,0.443987,0.439714,0.487282,0.614145,0.746810,0.489232,0.467438,0.433062,0.404051,0.488985,0.617498,0.551244,0.446972,0.414444,0.452678,0.497552,0.631603,0.741104,0.814298,0.665651,0.618957,0.652682,0.589925,0.656079,0.863431,0.934385
3,HOBBIES_1_004_CA_1_validation,1.953827,1.650163,1.582948,1.449167,1.882337,2.292001,2.373877,1.683620,1.729427,1.583586,1.363456,1.760662,2.513070,2.371239,1.990482,1.594354,1.684561,1.622765,1.804648,2.274500,2.554568,1.955439,1.656123,1.546695,1.504117,1.803702,2.746094,2.708756
4,HOBBIES_1_005_CA_1_validation,0.973892,0.852962,0.938600,1.009142,1.059734,1.381942,1.712063,1.207382,1.142862,1.006309,0.848956,1.011457,1.392919,1.202308,1.083366,0.925229,0.934676,0.935744,1.087515,1.349977,1.321237,0.957902,0.943921,0.933255,0.931469,1.061367,1.397413,1.589579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
